Buisness Scenario: Build a model that can learn language pattern from text and generate new sentences ,predict the next words when some words are provided to it.

In [1]:

pip install nltk

In [2]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Import all the necessary libraries
from warnings import filterwarnings
filterwarnings(action='ignore')

# Preprocess the text
from tensorflow.keras.preprocessing.text import Tokenizer # creates work tokens, number sequences
from tensorflow.keras.utils import pad_sequences # padding
from keras.preprocessing import sequence
# Model
from keras.models import Sequential
from keras.layers import Input,Dense,LSTM,Embedding


LOad the dataset:Data Preparation : -loaded the data set- shakesphare's plays . -Tokenized texts into words and created a vocabulary. -converted text to sequence of word indices.

In [4]:
import pandas as pd
data = pd.read_csv('/content/Shakespeare_data.csv')

print(data.head())

   Dataline      Play  PlayerLinenumber ActSceneLine         Player  \
0         1  Henry IV               NaN          NaN            NaN   
1         2  Henry IV               NaN          NaN            NaN   
2         3  Henry IV               NaN          NaN            NaN   
3         4  Henry IV               1.0        1.1.1  KING HENRY IV   
4         5  Henry IV               1.0        1.1.2  KING HENRY IV   

                                          PlayerLine  
0                                              ACT I  
1                       SCENE I. London. The palace.  
2  Enter KING HENRY, LORD JOHN OF LANCASTER, the ...  
3             So shaken as we are, so wan with care,  
4         Find we a time for frighted peace to pant,  


In [5]:
data[:15]

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil
8,9,Henry IV,1.0,1.1.6,KING HENRY IV,Shall daub her lips with her own children's bl...
9,10,Henry IV,1.0,1.1.7,KING HENRY IV,"Nor more shall trenching war channel her fields,"


We are using Keras Tokenizer here. Few points are discussed below related to fit_on_texts method and texts_to_sequences method: tokenizer.fit_on_texts(texts) : Builds the word index (vocabulary) from your dataset.

It looks at the list of texts you give it. It assigns a unique integer index to each unique word. The most frequent word gets index 1, the next most frequent 2, and so on. Index 0 is reserved for padding.

Example: texts = ["The wizard cast a spell", "The spell was powerful"] tokenizer.fit_on_texts(texts) print(tokenizer.word_index) Output: {'the': 1, 'spell': 2, 'wizard': 3, 'cast': 4, 'a': 5, 'was': 6, 'powerful': 7} tokenizer.texts_to_sequences(texts): Converts your texts into sequences of integers based on the word index created by fit_on_texts.

Example: sequences = tokenizer.texts_to_sequences(["The wizard cast a spell"])
print(sequences) Output: [[1, 3, 4, 5, 2]]


In [6]:
max_features= 1000
# Intializing the tokenizer
tokenizer = Tokenizer(num_words = max_features)
# Fit on texts - pass the data
tokenizer.fit_on_texts(data['PlayerLine'])
word_ind = tokenizer.word_index
vocab_size = min (len(word_ind), max_features) + 1  # +1 for padding
# to reserve for padding
print(word_ind)




{'the': 1, 'and': 2, 'i': 3, 'to': 4, 'of': 5, 'a': 6, 'you': 7, 'my': 8, 'in': 9, 'that': 10, 'is': 11, 'not': 12, 'me': 13, 'it': 14, 'with': 15, 'for': 16, 'be': 17, 'his': 18, 'your': 19, 'this': 20, 'he': 21, 'but': 22, 'have': 23, 'as': 24, 'thou': 25, 'him': 26, 'will': 27, 'so': 28, 'what': 29, 'her': 30, 'thy': 31, 'all': 32, 'no': 33, 'do': 34, 'by': 35, 'shall': 36, 'if': 37, 'are': 38, 'we': 39, 'thee': 40, 'our': 41, 'on': 42, 'now': 43, 'good': 44, 'lord': 45, 'from': 46, 'sir': 47, 'at': 48, 'come': 49, 'they': 50, 'enter': 51, 'or': 52, 'would': 53, 'she': 54, 'more': 55, 'which': 56, 'well': 57, 'was': 58, 'o': 59, 'how': 60, 'am': 61, 'then': 62, 'here': 63, 'let': 64, 'their': 65, 'them': 66, 'love': 67, 'when': 68, 'hath': 69, 'than': 70, 'man': 71, 'there': 72, 'like': 73, 'one': 74, "i'll": 75, 'an': 76, 'go': 77, 'upon': 78, 'know': 79, 'us': 80, 'say': 81, 'may': 82, 'make': 83, 'did': 84, 'were': 85, 'yet': 86, 'should': 87, 'king': 88, 'must': 89, 'why': 90, '

In [7]:
reverse_word_index = {value:key for key,value in word_ind.items()}
reverse_word_index

{1: 'the',
 2: 'and',
 3: 'i',
 4: 'to',
 5: 'of',
 6: 'a',
 7: 'you',
 8: 'my',
 9: 'in',
 10: 'that',
 11: 'is',
 12: 'not',
 13: 'me',
 14: 'it',
 15: 'with',
 16: 'for',
 17: 'be',
 18: 'his',
 19: 'your',
 20: 'this',
 21: 'he',
 22: 'but',
 23: 'have',
 24: 'as',
 25: 'thou',
 26: 'him',
 27: 'will',
 28: 'so',
 29: 'what',
 30: 'her',
 31: 'thy',
 32: 'all',
 33: 'no',
 34: 'do',
 35: 'by',
 36: 'shall',
 37: 'if',
 38: 'are',
 39: 'we',
 40: 'thee',
 41: 'our',
 42: 'on',
 43: 'now',
 44: 'good',
 45: 'lord',
 46: 'from',
 47: 'sir',
 48: 'at',
 49: 'come',
 50: 'they',
 51: 'enter',
 52: 'or',
 53: 'would',
 54: 'she',
 55: 'more',
 56: 'which',
 57: 'well',
 58: 'was',
 59: 'o',
 60: 'how',
 61: 'am',
 62: 'then',
 63: 'here',
 64: 'let',
 65: 'their',
 66: 'them',
 67: 'love',
 68: 'when',
 69: 'hath',
 70: 'than',
 71: 'man',
 72: 'there',
 73: 'like',
 74: 'one',
 75: "i'll",
 76: 'an',
 77: 'go',
 78: 'upon',
 79: 'know',
 80: 'us',
 81: 'say',
 82: 'may',
 83: 'make',
 8

Text on sequences

In [8]:
vocab_size

1001

In [9]:
input_sequences = []
for line in data['PlayerLine']:   #iterate through each row in the column
  token_list = tokenizer.texts_to_sequences([line])[0]   #tokenize the line

  for i in range(1,len(token_list)):   # for i in range(1,4)
    n_gram_sequence = token_list[:i+1]    # token_list[0:2] = 0th index,1st index
    # token_list[0:3] = 0th index,1st index,2nd index
    # token_list[0:4] = 0th index, 1st index, 2nd index, 3rd index
    input_sequences.append(n_gram_sequence) # [[4,5],[4,5,6],[4,5,6,7]]
print(input_sequences[:15])

[[315, 3], [135, 3], [135, 3, 802], [135, 3, 802, 1], [135, 3, 802, 1, 571], [51, 88], [51, 88, 338], [51, 88, 338, 45], [51, 88, 338, 45, 397], [51, 88, 338, 45, 397, 5], [51, 88, 338, 45, 397, 5, 1], [51, 88, 338, 45, 397, 5, 1, 879], [51, 88, 338, 45, 397, 5, 1, 879, 5], [51, 88, 338, 45, 397, 5, 1, 879, 5, 47], [51, 88, 338, 45, 397, 5, 1, 879, 5, 47, 2]]


padding to bring all the ngram tokens to same size. to bring all the setences to same length

max([90,89,77])
     

In [10]:

max_length = max([len(lines) for lines in input_sequences])


input_sequences = sequence.pad_sequences(input_sequences,maxlen=max_length,truncating ='post')
input_sequences[:15]



array([[  0,   0,   0, ...,   0, 315,   3],
       [  0,   0,   0, ...,   0, 135,   3],
       [  0,   0,   0, ..., 135,   3, 802],
       ...,
       [  0,   0,   0, ...,   1, 879,   5],
       [  0,   0,   0, ..., 879,   5,  47],
       [  0,   0,   0, ...,   5,  47,   2]], dtype=int32)

In [11]:
print (max_length)

112


Separate x and y

In [12]:
x = input_sequences[:,:-1]
y = input_sequences[:,-1]



In [13]:


x[:10]

array([[  0,   0,   0, ...,   0,   0, 315],
       [  0,   0,   0, ...,   0,   0, 135],
       [  0,   0,   0, ...,   0, 135,   3],
       ...,
       [  0,   0,   0, ...,  51,  88, 338],
       [  0,   0,   0, ...,  88, 338,  45],
       [  0,   0,   0, ..., 338,  45, 397]], dtype=int32)

In [14]:
y[:10]

array([  3,   3, 802,   1, 571,  88, 338,  45, 397,   5], dtype=int32)

In [15]:
y.shape


(545466,)

y.shape
     

If you want to use categorical_crossentropy, then convert y into y cateogrical. y_cat = to_categorical(y)




Model Buidling: -Built an LSTM model for next - word prediction. -Trained the model on the sequence data .





In [16]:
from tensorflow.keras.utils import to_categorical
y_categorical = to_categorical(y,num_classes=vocab_size)# converts integer labels to categorical format                                                          #where total length here represents vocabulary size

model = Sequential()
# Provide the input as max length indicating total number of sentences
model.add(Input(shape=(max_length,)))# input shape
# Add the layers
model.add(Embedding(input_dim=vocab_size,output_dim=50,trainable=False)) # Embedding layers to represent words
model.add(LSTM(16,return_sequences=True,dropout=0.4)) # return sequences will provide the sequences to next LSTM layer
model.add(LSTM(8,dropout=0.3)) # LSTM layer for sequential learning
# Add output layer # predicts the probabilities for each word in the vocabulary
model.add(Dense(vocab_size, activation='softmax')) #dense and softmax layer for next word prediction



In [17]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [18]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)


In [22]:
nn = model.fit(x,y_categorical,validation_split=0.2,epochs=10,callbacks=[early_stop],batch_size=128)


Epoch 1/10
 493/3410 ━━━━━━━━━━━━━━━━━━━━ 6:26 132ms/step - accuracy: 0.0543 - loss: 5.5133

KeyboardInterrupt: 

Text Generation: -Used temperature sampling to control randomness. -Updated the input sequence with predicted words.

-experimented with the temperature values(example 1.0,1.5)-lowering the temperature makes the text more predictable.

In [24]:
import numpy as np
from nltk.tokenize import word_tokenize

def generate_text(user_text,next_words=60,temperature = 1.2):
    # preparing our user text ready for the model, initialising token list
    token_list = tokenizer.texts_to_sequences([user_text])[0]
    # padding the sequence
    token_list = pad_sequences([token_list],maxlen=max_length-1,padding = 'pre')
    generated_text = user_text
    # give the proceesed text to model for prediction of next words
    for _ in range(next_words):
        predicted_probs = model.predict(token_list,verbose=0)[0]
        predicted_probs = predicted_probs**(1/temperature)
        predicted_probs/= predicted_probs.sum()
        output_word_idx = np.random.choice(len(predicted_probs), p = predicted_probs) # it gives you the index of next word
        output_word = tokenizer.index_word[output_word_idx] # convert the index into word using index_word
        generated_text += ' ' + output_word
        token_list = np.append(token_list[0,1:],output_word_idx).reshape(1,max_length-1)
    return generated_text



In [26]:
seed1 ="To be or not to be"

generated_output =generate_text(seed1,next_words=60,temperature = 1.2)
print(generated_output)



To be or not to be we the of for dies thy far eye to the to i' my queen you he praise this neither purse their of all false please been opinion for send hands he love thee we'll well by made is two grace weep thee never go is thee heart too cannot it but is o' he this comfort so for these old


In [31]:
seed2 = "Romeo , Romeo, wherefore art thou"

generated_output= generate_text(seed2,next_words=60,temperature = 1.2)
print(generated_output)


Romeo , Romeo, wherefore art thou not there holy comes please or too love and slave time humour thine here on at is he clarence next is else each me i from off other from indeed pass break to thing eyes hate an in a besides me my have face doth to show and fashion not i sweet in the other till those be villain takes


In [32]:
seed3 = "Shall I compare thee to a summer's day "
generated_output =generate_text(seed3,next_words=60,temperature = 1.2)
print(generated_output)


Shall I compare thee to a summer's day  passion claudio father strange land close not than he give much i have but makes i am less and him signior your him still same your out but break young be presence help let kiss a eye you so it as years would spirit company as hath forgot shape to bosom doubt power what himself did life home in by


Data Preparation : -loaded the data set- shakesphare's plays . -Tokenized texts into words and created a vocabulary. -converted text to sequence of word indices.

Model Training: -Built an LSTM model for next - word prediction. -Trained the model on the sequence data .

Text Generation: -Used temperature sampling to control randomness. -Updated the input sequence with predicted words.

-experimented with the temperature values(example 1.0,1.5)-lowering the temperature makes the text more predictable.

Obstacles: -Repetition : Model often gets stuck in loops("of the the of the "). -Poetic Flair: Output doesn't always capture Shakespeare's style (example rhymes ,meter).